# Recommendation system sử dụng dữ liệu của foody

In [1]:
#library
import pandas as pd
import surprise
from surprise import SVD, Dataset, Reader, evaluate, print_perf, accuracy, GridSearch, KNNBaseline

# Đọc dữ liệu

In [3]:
data = pd.read_pickle('/home/zero/jupyter/Foody review/Foody sentiment/foody review.pkl')
data.head()

,user_link,user_name,brand_name,brand_link,review_content,score
0,https://www.foody.vn/thanh-vien/taiyo,Sunny,El Sol - Meat & Wine - Võ Thị Sáu,https://www.foody.vn/ho-chi-minh/el-sol-meat-w...,Quán steak hiếm hoi mà mình cực kì ưng ý từ lâ...,8.6
1,https://www.foody.vn/thanh-vien/viola_0309_tv,Nguyên Khánh,Busan Korean Food - Món Hàn Quốc - Đinh Tiên H...,https://www.foody.vn/ho-chi-minh/busan-korean-...,Vị trí dễ tìm. Giữ xe rất nhiệt tình. Dắt xe v...,7.4
2,https://www.foody.vn/thanh-vien/tuhao2509,Tú Hảo,TocoToco Bubble Tea - Cộng Hòa,https://www.foody.vn/ho-chi-minh/tocotoco-bubb...,"Địa điểm quán dễ tìm. Không gian cũng rộng, có...",6.4
3,https://www.foody.vn/thanh-vien/zip_zilip,Vy Nhật,Le Castella Viet Nam - Bánh Bông Lan Đài Loan ...,https://www.foody.vn/ho-chi-minh/le-castella-v...,Thấy bánh này đang hot rần rần mình cũng gọi t...,7.2
4,https://www.foody.vn/thanh-vien/tien97,Tien97,Tabletop - Boardgame & Coffee,https://www.foody.vn/ho-chi-minh/tabletop-boar...,Mỗi lần nghĩ tới boardgame thì sẽ nghĩ tới vô ...,10.0


In [4]:
review = pd.DataFrame(columns = ['user', 'item', 'rating'])
review['user'] = data['user_link'].apply(lambda x: x.replace('https://www.foody.vn/thanh-vien/', ''))
review['item'] = data['brand_link']
review['rating'] = data['score']

In [5]:
review.head()

,user,item,rating
0,taiyo,https://www.foody.vn/ho-chi-minh/el-sol-meat-w...,8.6
1,viola_0309_tv,https://www.foody.vn/ho-chi-minh/busan-korean-...,7.4
2,tuhao2509,https://www.foody.vn/ho-chi-minh/tocotoco-bubb...,6.4
3,zip_zilip,https://www.foody.vn/ho-chi-minh/le-castella-v...,7.2
4,tien97,https://www.foody.vn/ho-chi-minh/tabletop-boar...,10.0


In [6]:
review.shape

(174379, 3)

# Tạo hàm gợi ý địa điểm cho user

In [7]:
def build_predictions(local, scale = 10):
    #Lọc các đánh giá tại tỉnh thành
    review_local = review[review.item.str.contains(local)]
    
    #load dataset
    reader = Reader(rating_scale=(1, scale))
    dataset = Dataset.load_from_df(review_local, reader = reader)
    
    #Sử dụng gridsearch để tối ưu
    param_grid = {'n_epochs': [5, 10], 
                  'lr_all': [0.002, 0.005],
                  'reg_all': [0.4, 0.6]}
    grid_search = GridSearch(SVD, param_grid, measures=['RMSE'])
    
    dataset.split(n_folds=5)
    grid_search.evaluate(dataset)
    params = grid_search.best_params['RMSE']
    
    #Train model
    trainset = dataset.build_full_trainset()
    algo = SVD(lr_all = params['lr_all'],
               n_epochs = params['n_epochs'],
               reg_all = params['reg_all'])
    algo.train(trainset)
    
    #predictions
    testset = trainset.build_anti_testset()
    predictions = algo.test(testset)
    
    return(trainset, predictions)

In [8]:
def print_recommend_place(uids, n=5):  
    recommend_item = pd.DataFrame(columns = ['item', 'est'])
    
    for uid, iid, true_r, est, _ in predictions:
        if uid == uids:
            recommend_item = recommend_item.append(pd.DataFrame({'item': [iid], 'est': [est]}))
    
    recommend_item = recommend_item.sort_values(by = 'est', ascending= False)
    
    print('Gợi ý địa điểm cho ', uids)
    for items in recommend_item['item'][0:n]:
        print(data[data.brand_link == items]['brand_name'].values[0])

# Tạo hàm gợi ý địa điểm dựa vào một địa điểm cụ thể

In [23]:
def recommend_place(item, k = 5):   
    inner_iids = knn.get_neighbors(train_set.to_inner_iid(item), k=k)
    print('Bạn cũng có thể muốn tới các địa sau:' )
    for ids in inner_iids:
        print(data[data.brand_link.str.contains(train_set.to_raw_iid(ids))]['brand_name'].values[0])

# Áp dụng cho Hồ Chí Minh

In [229]:
%%time
train_set, predictions = build_predictions('ho-chi-minh')

[{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}, {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}, {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}, {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}, {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}, {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}, {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}, {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}]
------------
Parameters combination 1 of 8
params:  {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}
------------
Mean RMSE: 1.4414
------------
------------
Parameters combination 2 of 8
params:  {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}
------------
Mean RMSE: 1.4434
------------
------------
Parameters combination 3 of 8
params:  {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}
------------
Mean RMSE: 1.4246
------------
------------
Parameters combination 4 of 8
params:  {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}
------------
Mean RMSE: 1.4273
------------
------------
Pa

In [248]:
sim_options = {'name': 'pearson_baseline', 'user_based': False}
knn = KNNBaseline(sim_options=sim_options)
knn.train(train_set) 

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


## Gợi ý địa điểm cho user

### user tien97

In [275]:
review[review.user == 'tien97']

,user,item,rating
4,tien97,https://www.foody.vn/ho-chi-minh/tabletop-boar...,10.0


In [279]:
print_recommend_place('tien97')

Gợi ý địa điểm cho  tien97
An Nam Quán - Gà Lên Mâm - Nguyễn Thái Bình
Cafe Palan - Phan Xích Long
Haru Sushi
Starbucks Coffee - Thảo Điền
Poke Saigon


## user tuhao2509

In [280]:
review[review.user == 'tuhao2509']

,user,item,rating
2,tuhao2509,https://www.foody.vn/ho-chi-minh/tocotoco-bubb...,6.4
78647,tuhao2509,https://www.foody.vn/dong-nai/houjicha-tra-sua...,6.6
154525,tuhao2509,https://www.foody.vn/ho-chi-minh/texas-chicken...,8.0
162203,tuhao2509,https://www.foody.vn/ho-chi-minh/tocotoco-cao-...,6.2
165086,tuhao2509,https://www.foody.vn/ho-chi-minh/tra-sua-bobap...,7.0


In [281]:
print_recommend_place('tuhao2509')

Gợi ý địa điểm cho  tuhao2509
Haru Sushi
An Nam Quán - Gà Lên Mâm - Nguyễn Thái Bình
Starbucks Coffee - Thảo Điền
Oven Maru Chicken - Ẩm Thực Hàn Quốc
Dallas Cakes & Coffee - 3 Tháng 2


### user zip_zilip

In [285]:
review[review.user == 'zip_zilip'].head(10)

,user,item,rating
3,zip_zilip,https://www.foody.vn/ho-chi-minh/le-castella-v...,7.2
5,zip_zilip,https://www.foody.vn/ho-chi-minh/gogi-house-qu...,7.2
46,zip_zilip,https://www.foody.vn/ho-chi-minh/an-vat-muoi-t...,6.8
23653,zip_zilip,https://www.foody.vn/lam-dong/banh-can-cay-bo,8.0
41095,zip_zilip,https://www.foody.vn/ho-chi-minh/com-ga-xoi-mo...,5.6
42153,zip_zilip,https://www.foody.vn/ho-chi-minh/com-tam-ba-ghien,7.8
42156,zip_zilip,https://www.foody.vn/ho-chi-minh/com-tam-ba-ghien,7.6
42157,zip_zilip,https://www.foody.vn/ho-chi-minh/com-tam-ba-ghien,7.2
51584,zip_zilip,https://www.foody.vn/ho-chi-minh/ding-tea-cong...,5.4
59905,zip_zilip,https://www.foody.vn/ho-chi-minh/anh-tu-ga-nuong,7.4


In [284]:
print_recommend_place('zip_zilip')

Gợi ý địa điểm cho  zip_zilip
Haru Sushi
An Nam Quán - Gà Lên Mâm - Nguyễn Thái Bình
Starbucks Coffee - Thảo Điền
Oven Maru Chicken - Ẩm Thực Hàn Quốc
Dallas Cakes & Coffee - 3 Tháng 2


## Gợi ý địa điểm dựa vào một địa điểm cụ thể

In [286]:
recommend_place('https://www.foody.vn/ho-chi-minh/busan-korean-food-mon-an-han')

Bạn cũng có thể muốn tới các địa sau:
El Sol - Meat & Wine - Võ Thị Sáu
TocoToco Bubble Tea - Cộng Hòa
Le Castella Viet Nam - Bánh Bông Lan Đài Loan - Ngô Đức Kế
Tabletop - Boardgame & Coffee
Gogi House - Thịt Nướng Hàn Quốc - Lê Văn Sỹ


In [287]:
recommend_place('https://www.foody.vn/ho-chi-minh/an-vat-ca-vien-chien')

Bạn cũng có thể muốn tới các địa sau:
Tàu Hủ Xe Lam - Cách Mạng Tháng 8
Mì Ý Double B
Phúc Long Coffee & Tea House - Nowzone
Koi Thé Café - Ngô Đức Kế
Texas Chicken - Phan Xích Long


In [288]:
recommend_place('https://www.foody.vn/ho-chi-minh/chilli-thai-mac-thi-buoi')

Bạn cũng có thể muốn tới các địa sau:
Bánh Canh Ghẹ Ba Sạch
Friends By Pontian - Mì Singapore - Nguyễn Thái Học
Cơm Tấm Ba Ghiền
Út Cà Mau - Đặc Sản Cua Biển
Meet Fresh - Ngô Đức Kế


# Áp dụng cho Hà Nội

In [11]:
%%time
train_set, predictions = build_predictions('ha-noi')

[{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}, {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}, {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}, {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}, {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}, {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}, {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}, {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}]
------------
Parameters combination 1 of 8
params:  {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}
------------
Mean RMSE: 1.2071
------------
------------
Parameters combination 2 of 8
params:  {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}
------------
Mean RMSE: 1.2080
------------
------------
Parameters combination 3 of 8
params:  {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}
------------
Mean RMSE: 1.1845
------------
------------
Parameters combination 4 of 8
params:  {'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}
------------
Mean RMSE: 1.1869
------------
------------
Pa

In [13]:
sim_options = {'name': 'pearson_baseline', 'user_based': False}
knn = KNNBaseline(sim_options=sim_options)
knn.train(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


## Gợi ý địa điểm cho user

### user nhung.nguyen-phuong

In [19]:
review[review.user == 'nhung.nguyen-phuong'].head(10)

,user,item,rating
445,nhung.nguyen-phuong,https://www.foody.vn/ha-noi/cho-tet-thu-huong,7.8
31274,nhung.nguyen-phuong,https://www.foody.vn/ha-noi/chao-suon-dong-xuan,7.4
38101,nhung.nguyen-phuong,https://www.foody.vn/ha-noi/co-lien-che-da-nan...,7.0
58311,nhung.nguyen-phuong,https://www.foody.vn/ha-noi/food-center-o-cho-dua,7.8
59196,nhung.nguyen-phuong,https://www.foody.vn/ha-noi/funny-monkey-xa-dan,7.0
62922,nhung.nguyen-phuong,https://www.foody.vn/ha-noi/gimbab-han-quoc-ng...,7.4
72330,nhung.nguyen-phuong,https://www.foody.vn/ha-noi/hera-restaurant-am...,7.8
72333,nhung.nguyen-phuong,https://www.foody.vn/ha-noi/hera-restaurant-am...,8.0
79990,nhung.nguyen-phuong,https://www.foody.vn/ha-noi/huyen-vi-sinh-to-c...,8.0
80000,nhung.nguyen-phuong,https://www.foody.vn/ha-noi/huyen-vi-sinh-to-c...,8.6


In [20]:
print_recommend_place('nhung.nguyen-phuong')

Gợi ý địa điểm cho  nhung.nguyen-phuong
Phớ Đê - Tào Phớ Gây Thương Nhớ
Cô Quỳnh - Lẩu & Nướng
HealthWich Vietnam - Lương Ngọc Quyến
Fraicheur - Nước Ép Cold Pressed Nguyên Chất
JW Café - JW Marriott Hotel Hanoi


## Gợi ý địa điểm dựa vào một địa điểm cụ thể

In [24]:
recommend_place('https://www.foody.vn/ha-noi/cho-tet-thu-huong')

Bạn cũng có thể muốn tới các địa sau:
Paris Gâteaux - Giảng Võ
Nhà Hàng BBQ Yukssam
Cora Cafe - Hòa Mã
Tiệm Bánh Nhọ Nồi - Phó Đức Chính
Wanna Waffle - Đinh Tiên Hoàng
